| 컬럼명 | 설명 | 값 의미 | 비고 |
|:---:|:---|:---|:---|
| **Churn** | 고객 이탈 여부 | **1**: 이탈 (Yes), **0**: 유지 (No) |
| **Age** | 고객의 나이 | 정수형 (int) | |
| **Region** | 거주 지역 | 범주형 (str) | Seoul, Metro, Other |
| **Tenure_Months** | 서비스 가입 기간 | 정수형 (int) | 개월 수 단위 |
| **Total_Usage_GB** | 최근 데이터 사용량 | 실수형 (float) | GB 단위 |
| **Support_Tickets_3M**| 최근 3개월 CS 문의 건수 | 정수형 (int) | 고객 불만 지표 |
| **NPS_Score** | 고객 만족도 점수 | 정수형 (int) | -100 ~ 100 점 |
| **Monthly_Fee** | 월 납부 요금 | 실수형 (float) | |
| **Late_Payments_6M** | 최근 6개월 연체 횟수 | 정수형 (int) | |
| **Contract** | 계약 형태 | 범주형 (str) | Month-to-month, 1-year, 2-year |
| **AutoPay** | 자동결제 여부 | 이진형 (bool) | 1: 사용, 0: 미사용 |
| **Internet_Type** | 회선 종류 | 범주형 (str) | Fiber, DSL, 5G |
| **Has_Addon** | 부가서비스(0/1)| 정수형(int) | |

In [ ]:
import pandas as pd

df = pd.read_csv('../../data/churn_train.csv')
df.head()

In [ ]:
df.info()

In [ ]:
import koreanize_matplotlib # noqa: F401
import matplotlib.pyplot as plt

t = df['Churn'].value_counts()
labels = ['유지', '이탈']

fig, ax = plt.subplots()
ax.pie(t, labels=labels, autopct='%1.1f%%')


In [ ]:
import seaborn as sns

selected_cols = ['Age', 'Tenure_Months', 'Monthly_Fee', 'Total_Usage_GB', 'Churn']

sns.pairplot(data=df[selected_cols], hue='Churn', plot_kws={'alpha': 0.4})

In [ ]:
selected_cols = ['Support_Tickets_3M', 'NPS_Score', 'Late_Payments_6M', 'Contract', 'Churn']

sns.pairplot(data=df[selected_cols], hue='Churn', plot_kws={'alpha': 0.4})

In [ ]:
col = ['Age', 'Tenure_Months', 'Monthly_Fee', 'NPS_Score', 'Total_Usage_GB', 'Support_Tickets_3M', 'Late_Payments_6M']
n = len(col)

fig, ax = plt.subplots(n, 2, figsize=(12, 5 * n))

# plt.figure(figsize=(15, 10))
for i, col in enumerate(col):
    # (왼쪽) KDE Plot: 분포의 모양 비교
    sns.kdeplot(data=df, x=col, hue='Churn', fill=True, ax=ax[i, 0])
    plt.title(f'{col}에 따른 이탈 분포 (KDE)')
    
    # (오른쪽) Box Plot: 중앙값과 이상치 비교
    sns.boxplot(data=df, x='Churn', y=col, ax=ax[i, 1])
    plt.title(f'이탈 여부별 {col} 차이 (Box)')

plt.show()

In [ ]:
df

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OrdinalEncoder

encoder = OrdinalEncoder()
scaler = StandardScaler()

col = ["Contract", "Internet_Type", "Region", "Monthly_Fee"]
df[col] = encoder.fit_transform(df[col].copy())

df['Monthly_Fee'] = scaler.fit_transform(df[['Monthly_Fee']])

corr_matrix = df.corr()

plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, cmap='RdBu', fmt='.2f', linewidths=0.5)
plt.title('Feature Correlation with Churn')
plt.show()